# Tutorial on running backtesting

In [20]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from quantrl_lab.data import (
    DataSourceRegistry,
    DataProcessor,
    IndicatorRegistry,
)
from quantrl_lab.data.indicators.technical_indicators import *
from quantrl_lab.backtesting.runner import BacktestRunner
from quantrl_lab.custom_envs.stock import SingleStockTradingEnv, SingleStockEnvConfig
from quantrl_lab.custom_envs.stock.strategies.actions.types import StandardMarketActionStrategy
from quantrl_lab.custom_envs.stock.strategies.rewards import (
    PortfolioValueChangeReward,
    InvalidActionPenalty,
    TrendFollowingReward,
    HoldPenalty,
    WeightedCompositeReward
)
from quantrl_lab.custom_envs.stock.strategies.observations import PortfolioWithTrendObservation
from stable_baselines3 import PPO, A2C, SAC

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)


# Using DataSourceRegistry as the loader to extract different sources of data

Currently supports OLHCV data an Sentiment data from Alpaca primarily, 
working on integrating Alpha Vantage's data (format standardization required).

In [2]:
data_loader = DataSourceRegistry()

In [4]:
olhcv_df = data_loader.get_historical_ohlcv_data(
    symbols="MU",
    start="2022-01-01",
    end="2025-06-30",
    timeframe="1d",
)

Fetching historical data for MU from 2022-01-01 to 2025-06-30 with timeframe 1d

In [5]:
olhcv_df.head()

,Symbol,Timestamp,Open,High,Low,Close,Volume,Trade_count,VWAP,Date
0,MU,2022-01-03 05:00:00+00:00,93.9050,95.835,93.48,95.75,19907641.0,163032.0,95.028965,2022-01-03
1,MU,2022-01-04 05:00:00+00:00,95.6000,96.590,93.54,96.34,20892833.0,192840.0,95.386837,2022-01-04
2,MU,2022-01-05 05:00:00+00:00,95.3800,98.450,94.35,94.40,28880722.0,252641.0,96.712594,2022-01-05
3,MU,2022-01-06 05:00:00+00:00,95.1046,96.710,94.12,95.65,23737246.0,215101.0,95.715463,2022-01-06
4,MU,2022-01-07 05:00:00+00:00,96.0900,97.190,93.94,94.45,16618859.0,153941.0,95.030314,2022-01-07


In [6]:
news_df = data_loader.get_news_data(
    "MU",
    start="2022-01-01",
    end="2025-06-30",
)

Output()

✓ Total news items fetched: 2277

Current list of technical indicators supported:
- SMA
- EMA
- RSI
- MACD
- ATR
- BB
- STOCH
- OBV

Working on supporting more indicators

In [7]:
data_processor = DataProcessor(olhcv_data=olhcv_df, news_data=news_df)

# configuring the technical indicators with specific parameters
processed_data = data_processor.data_processing_pipeline(
    indicators=["SMA", "EMA", "RSI", "MACD", "ATR", "BB", "STOCH", "OBV"],
    # The following are optional, you could do
    # data_processor.append_technical_indicators(olhcv_df, indicators=["SMA", "EMA"])
    # and the default parameters will be used.
    
    # fillna_strategy="neutral",
    SMA_params={
        "window": 20,
    },
    EMA_params={"window": 9},
    RSI_params={"window": 7},
    MACD_params={"fast": 12, "slow": 26, "signal": 9},
    ATR_params={"window": 14},
    BB_params={"window": 20, "num_std": 2},
    STOCH_params={"k_window": 14, "d_window": 3, "smooth_k": 1},
    # OBV_params={},
)

Applying SMA with params: {'window': 20}

Applying EMA with params: {'window': 9}

Applying RSI with params: {'window': 7}

Applying MACD with params: {'fast': 12, 'slow': 26, 'signal': 9}

Applying ATR with params: {'window': 14}

Applying BB with params: {'window': 20, 'num_std': 2}

Applying STOCH with params: {'k_window': 14, 'd_window': 3, 'smooth_k': 1}

Applying OBV with params: {}

Device set to use cpu


✓ Sentiment analysis pipeline initialized with model: ProsusAI/finbert

In [8]:
processed_data.head()

,Open,High,Low,Close,Volume,Trade_count,VWAP,SMA_20,EMA_9,RSI_7,MACD_line_12_26,MACD_signal_9,MACD_histogram,ATR_14,BB_middle_20,BB_upper_20_2,BB_lower_20_2,BB_bandwidth_20,STOCH_%K_14,STOCH_%D_3,OBV,sentiment_score
0,78.960,82.31,78.02,82.27,25587263.0,177832.0,81.231750,89.4270,83.289881,38.546670,-4.056880,-2.957446,-1.099434,3.908338,89.4270,103.057519,75.796481,0.304841,29.887093,17.256112,29104168.0,0.615621
1,82.200,82.39,80.42,81.45,16677706.0,156159.0,81.282006,88.7120,82.921905,36.195812,-4.012427,-3.168442,-0.843985,3.769885,88.7120,102.445824,74.978176,0.309627,26.256365,24.249133,12426462.0,0.861282
2,82.920,84.70,82.54,84.51,21429140.0,174863.0,83.930710,88.1205,83.239524,49.582550,-3.687770,-3.272308,-0.415463,3.732750,88.1205,101.485089,74.755911,0.303325,39.805180,31.982879,33855602.0,0.000000
3,82.670,85.18,81.71,81.97,21519262.0,183404.0,83.199568,87.4990,82.985619,41.209527,-3.594005,-3.336647,-0.257358,3.713982,87.4990,100.789910,74.208090,0.303796,29.438613,31.833386,12336340.0,0.000000
4,81.222,82.19,79.84,81.17,16827299.0,153181.0,81.113986,86.7750,82.622495,38.801794,-3.543403,-3.377998,-0.165404,3.616555,86.7750,99.770654,73.779346,0.299525,27.655409,32.299734,-4490959.0,0.000000


## Comprehensive Backtesting Example

#### Train-test split and keeping them as dataframes. The environment will handle to price column detection and numpy conversion

In [9]:
train_size = int(len(processed_data) * 0.8)
train_data_df = processed_data[:train_size]  # Keep as DataFrame
test_data_df = processed_data[train_size:]   # Keep as DataFrame

#### Strategy instances

In [10]:
# Create strategy instances
action_strategy = StandardMarketActionStrategy()
observation_strategy = PortfolioWithTrendObservation()

# Create composite reward strategy
portfolio_reward = PortfolioValueChangeReward()
invalid_penalty = InvalidActionPenalty(penalty=-1.0)
trend_reward = TrendFollowingReward()
hold_penalty = HoldPenalty(penalty=-0.5)

# Multiple reward strategies with different weights
# These weights can be adjusted based on the desired emphasis on each strategy
reward_strategies = {
    "balanced": WeightedCompositeReward(
        strategies=[portfolio_reward, invalid_penalty, trend_reward, hold_penalty],
        weights=[1.0, 2.0, 0.3, 0.1]
    ),
    "conservative": WeightedCompositeReward(
        strategies=[portfolio_reward, invalid_penalty, trend_reward, hold_penalty],
        weights=[1.0, 3.0, 0.2, 0.05]
    ),
    "aggressive": WeightedCompositeReward(
        strategies=[portfolio_reward, invalid_penalty, trend_reward, hold_penalty],
        weights=[0.8, 1.5, 0.6, 0.3]
    )
}

In [23]:
# Create multiple environment configurations for comprehensive backtesting
env_configs = {
    "standard": {
        'train_env_factory': lambda: SingleStockTradingEnv(
            data=train_data_df,
            config=SingleStockEnvConfig(
                initial_balance=100000.0,
                transaction_cost_pct=0.001,  # Standard fees
                slippage=0.0005,
                window_size=20,
                order_expiration_steps=5
            ),
            action_strategy=action_strategy,
            reward_strategy=reward_strategies["conservative"],
            observation_strategy=observation_strategy
        ),
        'test_env_factory': lambda: SingleStockTradingEnv(
            data=test_data_df,
            config=SingleStockEnvConfig(
                initial_balance=100000.0,
                transaction_cost_pct=0.001,
                slippage=0.0005,
                window_size=20,
                order_expiration_steps=5
            ),
            action_strategy=action_strategy,
            reward_strategy=reward_strategies["conservative"],
            observation_strategy=observation_strategy
        )
    },
    "low_cost": {
        'train_env_factory': lambda: SingleStockTradingEnv(
            data=train_data_df,
            config=SingleStockEnvConfig(
                initial_balance=100000.0,
                transaction_cost_pct=0.0,  # No fees
                slippage=0.001,
                window_size=20,
                order_expiration_steps=5
            ),
            action_strategy=action_strategy,
            reward_strategy=reward_strategies["conservative"], 
            observation_strategy=observation_strategy
        ),
        'test_env_factory': lambda: SingleStockTradingEnv(
            data=test_data_df,
            config=SingleStockEnvConfig(
                initial_balance=100000.0,
                transaction_cost_pct=0.0,
                slippage=0.001,
                window_size=20,
                order_expiration_steps=5
            ),
            action_strategy=action_strategy,
            reward_strategy=reward_strategies["conservative"], 
            observation_strategy=observation_strategy
        )
    }
}

print("✅ Created environment configs:")
print(f"   Environment names: {list(env_configs.keys())}")
print("   Each environment has train_env_factory and test_env_factory")

✅ Created environment configs:
   Environment names: ['standard', 'low_cost']
   Each environment has train_env_factory and test_env_factory


In [24]:
# Also create a single environment config for individual experiments
single_env_config = env_configs["standard"]  # Use the standard configuration

print("✅ Single environment config created for individual experiments")
print("   Use single_env_config for: run_single_experiment, run_algorithm_comparison, run_preset_comparison")
print("   Use env_configs for: run_comprehensive_backtest, run_environment_comparison")

✅ Single environment config created for individual experiments
   Use single_env_config for: run_single_experiment, run_algorithm_comparison, run_preset_comparison
   Use env_configs for: run_comprehensive_backtest, run_environment_comparison


In [25]:
# Single experiment using custom configuration

runner = BacktestRunner(verbose=True)

# TODO: read up the algorithms again 

custom_config = BacktestRunner.create_custom_config(
    SAC,
    learning_rate=0.0001,           # Lower learning rate
    batch_size=256,
    gamma=0.995,                    # Higher gamma for long-term focus
    tau=0.01,                       # Slower soft updates
    ent_coef=0.2,                   # Higher entropy for more exploration
    target_update_interval=2,       # Less frequent target updates
    train_freq=4,                   # Train every 4 steps
    gradient_steps=1,
    buffer_size=1000000,             
    policy_kwargs={
        'net_arch': [256, 256],      
        'activation_fn': nn.ReLU  # not that you cannot use str here, it must be a callable
    }
)

# Run single experiment using the single_env_config
results = runner.run_single_experiment(
    SAC,          # Algorithm to use
    single_env_config,     # Use the single environment config we created
    config=custom_config,  # Custom algorithm configuration
    total_timesteps=50000,
    num_eval_episodes=5
)

print(f"✅ Single experiment completed!")

============================================================

RUNNING SINGLE EXPERIMENT: SAC

Using custom configuration with 10 parameters

Custom params: ['learning_rate', 'batch_size', 'gamma', 'tau', 'ent_coef', 'target_update_interval', 'train_freq', 
'gradient_steps', 'buffer_size', 'policy_kwargs']

Timesteps: 50,000

...

          Experiment Results           
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┓
┃ Metric             ┃ Train ┃ Test   ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━┩
│ Average Return (%) │ 9.47% │ -3.95% │
│ Average Reward     │ 3.13  │ 2.50   │
└────────────────────┴───────┴────────┘

Action Statistics:

                    Action Distribution                     
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┓
┃ Action     ┃ Train Count ┃ Train % ┃ Test Count ┃ Test % ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━┩
│ Buy        │          10 │    0.3% │          0 │   0.0% │
│ Hold       │           5 │    0.2% │          5 │   0.7% │
│ LimitBuy   │        3225 │   97.3% │        695 │  92.7% │
│ LimitSell  │           5 │    0.2% │         15 │   2.0% │
│ Sell       │          65 │    2.0% │         30 │   4.0% │
│ StopLoss   │           5 │    0.2% │          0 │   0.0% │
│ TakeProfit │           0 │    0.0% │          5 │   0.7% │
└────────────┴─────────────┴─────────┴────────────┴────────┘

       Step Summary       
┏━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Phase    ┃ Total Steps ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Training │        3315 │
│ Testing  │         750 │
└──────────┴─────────────┘

✅ Single experiment completed!


In [26]:
# Step 1: Create custom configurations for PPO and SAC
# A2C will use the presets defined later
custom_configs = {
    'PPO': BacktestRunner.create_custom_config(
        PPO,
        learning_rate=0.001,
        n_steps=2048,
        batch_size=64,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.01
    ),
    'SAC': BacktestRunner.create_custom_config(
        SAC,
        learning_rate=0.0003,
        batch_size=256,
        gamma=0.99,
        tau=0.005,
        ent_coef='auto',
        target_update_interval=1,
        train_freq=1,
        policy_kwargs={
            'net_arch': [256, 256]
        }
    )
    # A2C not specified here, so it will use presets
}

# Step 2: Define presets for A2C (and as fallback)
presets = ["explorative"]

# Step 3: Run comprehensive backtest
algorithms = [PPO, A2C, SAC]

comprehensive_results = runner.run_comprehensive_backtest(
    algorithms=algorithms,
    env_configs=env_configs,
    presets=presets,           # A2C will use these presets
    custom_configs=custom_configs,  # PPO and SAC will use custom configs
    total_timesteps=50000,
    n_envs=4,
    num_eval_episodes=5
)

print("\n✅ Comprehensive backtest completed!")

====================================================================================================

COMPREHENSIVE BACKTESTING

Algorithms: ['PPO', 'A2C', 'SAC']

Environments: ['standard', 'low_cost']

Custom configs: ['PPO', 'SAC']

...

====================================================================================================

COMPREHENSIVE BACKTESTING SUMMARY

====================================================================================================

                                       Comprehensive Backtesting Results                                       
┏━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Algorithm ┃ Environment ┃ Preset      ┃ Train Return % ┃ Test Return % ┃ Top Train Action ┃ Top Test Action ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ PPO       │ standard    │ custom      │          0.00% │         0.00% │ Hold (100.0%)    │ Hold (100.0%)   │
│ PPO       │ low_cost    │ custom      │          0.00% │         0.91% │ Hold (99.8%)     │ Hold (99.3%)    │
│ A2C       │ standard    │ explorative │          0.00% │         0.00% │ Hold (100.0%)    │ Hold (100.0%)   │
│ A2C       │ low_cost    │ explorative │          0.00% │         0.00% │ Hold (100.0%)    │ Hold (100.0%)   │
│ SAC       │ standard    │ custom      │        -99.47% │        40.89% │ Buy (29.3%)      │ Hold (31.3%)    │
│ SAC       │ low_cost    │ custom      │        -27.87% │       -21.40% │ Buy (45.9%)      │ Buy (57.3%)     │
└───────────┴─────────────┴─────────────┴────────────────┴───────────────┴──────────────────┴─────────────────┘


✅ Comprehensive backtest completed!


In [ ]:
#TODO: include the following functions in a separate module for better organization
# These functions will handle printing the results in a comprehensive and user-friendly way
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.columns import Columns
from rich.text import Text

console = Console()

def print_comprehensive_results_summary(results):
    """
    Print a comprehensive summary of all results in a nicely formatted way.
    
    Args:
        results: The comprehensive_results dictionary from run_comprehensive_backtest
    """
    
    # Main title
    console.print("\n" + "="*80, style="bold blue")
    console.print("COMPREHENSIVE BACKTESTING RESULTS SUMMARY", style="bold blue", justify="center")
    console.print("="*80, style="bold blue")
    
    for algo_name, algo_results in results.items():
        # Algorithm header
        console.print(f"\n🤖 [bold cyan]{algo_name}[/bold cyan]")
        console.print("-" * 60, style="cyan")
        
        for env_name, env_results in algo_results.items():
            # Environment header
            console.print(f"\n  🌍 [bold yellow]{env_name.upper()} Environment[/bold yellow]")
            
            for config_name, result in env_results.items():
                if 'error' in result:
                    console.print(f"    ❌ [red]{config_name}: ERROR - {result['error']}[/red]")
                    continue
                
                # Create a panel for each configuration
                panel_content = create_result_panel_content(result, config_name)
                
                panel = Panel(
                    panel_content,
                    title=f"[bold]{config_name.upper()} Configuration[/bold]",
                    border_style="green" if result['test_avg_return_pct'] >= 0 else "red",
                    padding=(1, 2)
                )
                
                console.print(panel)

def create_result_panel_content(result, config_name):
    """Create formatted content for a result panel."""
    
    # Performance metrics
    train_return = result['train_avg_return_pct']
    test_return = result['test_avg_return_pct']
    
    train_color = "green" if train_return >= 0 else "red"
    test_color = "green" if test_return >= 0 else "red"
    
    # Create performance table
    perf_table = Table(show_header=True, header_style="bold magenta", box=None)
    perf_table.add_column("Metric", style="cyan", no_wrap=True)
    perf_table.add_column("Training", justify="right")
    perf_table.add_column("Testing", justify="right")
    
    perf_table.add_row(
        "Return %",
        f"[{train_color}]{train_return:.2f}%[/{train_color}]",
        f"[{test_color}]{test_return:.2f}%[/{test_color}]"
    )
    
    perf_table.add_row(
        "Avg Reward",
        f"{result['train_avg_reward']:.3f}",
        f"{result['test_avg_reward']:.3f}"
    )
    
    perf_table.add_row(
        "Reward Std",
        f"{result['train_reward_std']:.3f}",
        f"{result['test_reward_std']:.3f}"
    )
    
    # Action statistics
    train_actions = result.get('train_action_stats', {}).get('action_percentages', {})
    test_actions = result.get('test_action_stats', {}).get('action_percentages', {})
    
    action_info = ""
    if train_actions:
        top_train_action = max(train_actions, key=train_actions.get)
        top_train_pct = train_actions[top_train_action]
        action_info += f"🎯 Top Train Action: [bold]{top_train_action}[/bold] ({top_train_pct:.1f}%)\n"
    
    if test_actions:
        top_test_action = max(test_actions, key=test_actions.get)
        top_test_pct = test_actions[top_test_action]
        action_info += f"🎯 Top Test Action: [bold]{top_test_action}[/bold] ({top_test_pct:.1f}%)\n"
    
    # Configuration info
    config_info = ""
    if result.get('config'):
        config_info = f"⚙️  Custom Config: [dim]{len(result['config'])} parameters[/dim]\n"
        # Show key parameters
        key_params = ['learning_rate', 'batch_size', 'gamma', 'n_steps', 'tau', 'ent_coef']
        shown_params = []
        for param in key_params:
            if param in result['config']:
                shown_params.append(f"{param}={result['config'][param]}")
        if shown_params:
            config_info += f"   [dim]{', '.join(shown_params[:3])}[/dim]\n"
    else:
        config_info = f"⚙️  Preset: [dim]{result.get('preset', 'N/A')}[/dim]\n"
    
    # Training info
    training_info = f"🔄 Training: [dim]{result['total_timesteps']:,} timesteps[/dim]\n"
    training_info += f"📊 Episodes: [dim]{len(result['train_episodes'])} train, {len(result['test_episodes'])} test[/dim]\n"
    
    # Combine all content
    content = f"{config_info}{training_info}{action_info}\n{perf_table}"
    
    return content

def print_algorithm_comparison_table(results):
    """
    Print a comparison table across all algorithms and environments.
    """
    console.print("\n" + "="*100, style="bold magenta")
    console.print("ALGORITHM COMPARISON TABLE", style="bold magenta", justify="center")
    console.print("="*100, style="bold magenta")
    
    # Create comparison table
    table = Table(show_header=True, header_style="bold magenta")
    table.add_column("Algorithm", style="cyan", no_wrap=True)
    table.add_column("Environment", style="blue", no_wrap=True)
    table.add_column("Config", style="yellow", no_wrap=True)
    table.add_column("Test Return %", justify="right", style="bold")
    table.add_column("Train Return %", justify="right", style="bold")
    table.add_column("Test Reward", justify="right")
    table.add_column("Top Test Action", style="magenta")
    table.add_column("Status", justify="center")
    
    # Collect all results for sorting
    all_results = []
    for algo_name, algo_results in results.items():
        for env_name, env_results in algo_results.items():
            for config_name, result in env_results.items():
                all_results.append((algo_name, env_name, config_name, result))
    
    # Sort by test return percentage (descending)
    all_results.sort(key=lambda x: x[3].get('test_avg_return_pct', -float('inf')), reverse=True)
    
    for algo_name, env_name, config_name, result in all_results:
        if 'error' in result:
            table.add_row(
                algo_name, env_name, config_name,
                "[red]ERROR[/red]", "[red]ERROR[/red]", "[red]ERROR[/red]",
                "[red]ERROR[/red]", "❌"
            )
        else:
            test_return = result['test_avg_return_pct']
            train_return = result['train_avg_return_pct']
            
            test_color = "green" if test_return >= 0 else "red"
            train_color = "green" if train_return >= 0 else "red"
            
            # Get top test action
            test_actions = result.get('test_action_stats', {}).get('action_percentages', {})
            top_test_action = "N/A"
            if test_actions:
                top_action = max(test_actions, key=test_actions.get)
                top_pct = test_actions[top_action]
                top_test_action = f"{top_action} ({top_pct:.1f}%)"
            
            # Status emoji
            status = "✅" if test_return >= 0 else "❌"
            if test_return >= 5:
                status = "🚀"
            elif test_return >= 1:
                status = "📈"
            
            table.add_row(
                algo_name, env_name, config_name,
                f"[{test_color}]{test_return:.2f}%[/{test_color}]",
                f"[{train_color}]{train_return:.2f}%[/{train_color}]",
                f"{result['test_avg_reward']:.3f}",
                top_test_action,
                status
            )
    
    console.print(table)

def print_best_performers(results, top_n=3):
    """
    Print the top N best performing configurations.
    """
    console.print(f"\n🏆 [bold gold1]TOP {top_n} BEST PERFORMERS[/bold gold1]")
    console.print("="*50, style="gold1")
    
    # Collect and sort results
    all_results = []
    for algo_name, algo_results in results.items():
        for env_name, env_results in algo_results.items():
            for config_name, result in env_results.items():
                if 'error' not in result:
                    all_results.append((algo_name, env_name, config_name, result))
    
    # Sort by test return percentage
    all_results.sort(key=lambda x: x[3]['test_avg_return_pct'], reverse=True)
    
    for i, (algo_name, env_name, config_name, result) in enumerate(all_results[:top_n]):
        rank_emoji = ["🥇", "🥈", "🥉"][i] if i < 3 else f"{i+1}."
        
        test_return = result['test_avg_return_pct']
        color = "gold1" if i == 0 else "silver" if i == 1 else "orange3"
        
        console.print(f"{rank_emoji} [{color}]{algo_name}[/{color}] on [blue]{env_name}[/blue] "
                     f"with [yellow]{config_name}[/yellow]: [{color}]{test_return:.2f}%[/{color}]")

# Usage examples:
print_comprehensive_results_summary(comprehensive_results)
print_algorithm_comparison_table(comprehensive_results)
print_best_performers(comprehensive_results)

================================================================================

                                     COMPREHENSIVE BACKTESTING RESULTS SUMMARY                                     

================================================================================

🤖 PPO

------------------------------------------------------------

...

🏆 TOP 3 BEST PERFORMERS

==================================================

🥇 SAC on standard with custom: 40.89%

🥈 PPO on low_cost with custom: 0.91%

🥉 PPO on standard with custom: 0.00%

In [40]:
comprehensive_results["SAC"]["standard"]["custom"]["test_episodes"]

[{'episode': 1,
  'steps': 150,
  'final_value': np.float32(140893.12),
  'total_reward': 22.366318766810643,
  'initial_value': 100000.0,
  'actions_taken': {'Buy': 44,
   'Hold': 47,
   'LimitBuy': 20,
   'Sell': 28,
   'LimitSell': 6,
   'StopLoss': 4,
   'TakeProfit': 1},
  'detailed_actions': [{'step': 0,
    'action_type': 'Buy',
    'amount_pct': np.float32(0.92309976),
    'price_modifier': np.float32(1.0534039),
    'is_valid': True,
    'current_price': 97.7300033569336,
    'portfolio_value': 100069.63465206034,
    'reward': 0.0006963465206034016},
   {'step': 1,
    'action_type': 'Hold',
    'amount_pct': np.float32(0.8425568),
    'price_modifier': np.float32(0.970914),
    'is_valid': True,
    'current_price': 98.37000274658203,
    'portfolio_value': 100674.43407527811,
    'reward': -0.01895621433694001},
   {'step': 2,
    'action_type': 'LimitBuy',
    'amount_pct': np.float32(0.94922984),
    'price_modifier': np.float32(1.0650998),
    'is_valid': True,
    'curr

## BacktestRunner API Summary

### Method Overview

| Method | Parameter | Type | Description | Use Case |
|--------|-----------|------|-------------|----------|
| `run_single_experiment` | `env_config` | `Dict[str, Callable]` | Single environment config | Test one algorithm on one environment |
| `run_algorithm_comparison` | `env_config` | `Dict[str, Callable]` | Single environment config | Compare multiple algorithms on same environment |
| `run_preset_comparison` | `env_config` | `Dict[str, Callable]` | Single environment config | Compare different presets for same algorithm |
| `run_environment_comparison` | `env_configs` | `Dict[str, Dict[str, Callable]]` | Multiple environment configs | Test one algorithm across different environments |
| `run_comprehensive_backtest` | `env_configs` | `Dict[str, Dict[str, Callable]]` | Multiple environment configs | Full comparison: multiple algorithms × environments × presets |